In [1]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!python -m spacy download de_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 8.4 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [3]:
import os
import glob
import nemo.collections.asr as nemo_asr
import torch
import numpy as np
import ctc_decoders
import torchaudio
import soundfile as sf
import librosa

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [4]:
from processing.text_processor import preprocess_transcript_for_alignment, preprocess_transcript_for_sentence_split, split_to_sentences
from transcribing.stt_transcribe import get_stt_transcription, get_stt_probs

In [5]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(device)

1.11.0
cuda


## Read true transcript
Read the text from the complete protocol provided by the parliament of canton Schwyz. After reading the text process everything and split the whole text into its sentences. 

In [6]:
PATH_TO_TRANSCRIPT = 'data/text_processed_2021-02-24.txt'

In [7]:
with open(PATH_TO_TRANSCRIPT, encoding='utf-8') as f:
    transcript = f.read()
    transcript = preprocess_transcript_for_sentence_split(transcript)
    truth_sentences = split_to_sentences(transcript)

In [8]:
#print(truth_sentences)

## Retrieve all audio files to later match with the corresponding text

In [9]:
PATH_TO_AUDIO = '/data/voice/SZ_parliament/raw/parts'

In [10]:
audio_files = glob.glob(os.path.join(PATH_TO_AUDIO, '*.wav'))
#print(audio_files)

For testing use a test file randomly extracted from one of the audio files

In [11]:
# set a single pregenerated file for testing
TEST_FILE = ['test.wav']

## Transcription
Load and instantiate a pre-trained model from NVIDIA. In this case we use CTC decoding as output. 

In [12]:
# load model once
model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_de_quartznet15x5")

[NeMo I 2022-06-08 15:21:27 cloud:56] Found existing object /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo.
[NeMo I 2022-06-08 15:21:27 cloud:62] Re-using file from: /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo
[NeMo I 2022-06-08 15:21:27 common:747] Instantiating model from pre-trained checkpoint


[NeMo W 2022-06-08 15:21:28 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ä
    - ö
    - ü
    - ß
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-06-08 15:21:28 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    V

[NeMo I 2022-06-08 15:21:28 features:259] PADDING: 16
[NeMo I 2022-06-08 15:21:28 features:276] STFT using torch


[NeMo W 2022-06-08 15:21:29 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (WER). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    


[NeMo I 2022-06-08 15:21:30 save_restore_connector:209] Model EncDecCTCModel was successfully restored from /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo.


Retrieve the transcript of the given audio sequence

In [13]:
stt_output = get_stt_transcription(model, TEST_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
print(stt_output)

die  zu verfürtistehende mitte fahruftaupferde umeber aul um der nahmehölle zu unterstützte wo jetzt bringends weu sie hilf ruhet zerweine ich bil hihnen aus anders feiß das befehlende betroffenen unternahme personalhörchte tuchulzahrbezenschädigung oder repe duk croner werbsesots entschädigung und das siegt märz zwanzizwehn


For comparison we attach the actual true transcript of the given sequence

In [15]:
true_transcript = "die zur Verfügung stehenden Mittel fair aufgeteilt werden, um sämtliche Unternehmen unterstützen zu können, die jetzt dringend unsere Hilfe brauchen. Zu erwähnen ist auch, dass bei vielen betroffenen Unternehmen die Personalkosten durch die Kurzarbeitsentschädigung oder durch die Corona-Erwerbsersatzentschädigung seit März 2020 gedeckt"

In [16]:
print(true_transcript)

die zur Verfügung stehenden Mittel fair aufgeteilt werden, um sämtliche Unternehmen unterstützen zu können, die jetzt dringend unsere Hilfe brauchen. Zu erwähnen ist auch, dass bei vielen betroffenen Unternehmen die Personalkosten durch die Kurzarbeitsentschädigung oder durch die Corona-Erwerbsersatzentschädigung seit März 2020 gedeckt


In [17]:
!pip install biopython

Defaulting to user installation because normal site-packages is not writeable


## TODO
Implement the start and end point detection for each sentence, then pass it along to the sentence aligner

Get the probabilities for each token in the transcript

In [ ]:
def get_stt_probs(model, file):
    logits = model.transcribe(paths2audio_files=file, logprobs=True)[0]
    probs = softmax(logits)
    return probs

def get_stt_logits(model, file):
    logits = model.transcribe(paths2audio_files=file, logprobs=True)[0]
    return logits

def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])

In [ ]:
stt_probs = get_stt_probs(model, TEST_FILE)

In [ ]:
stt_logits = get_stt_logits(model, TEST_FILE)

In [ ]:
print(stt_logits)

## Timestep extraction
Define the duration of a timestep 

In [ ]:
# 40ms is duration of a timestep at output of the model
time_stride = 0.02

Get the model's alphabet and replace unknown token with 'space'

In [ ]:
labels = list(model.cfg.decoder.vocabulary) + ['blank']
labels[0] = 'space'

In [ ]:
print(labels)

In [ ]:
try:
    from plotly import graph_objects as go
except ModuleNotFoundError:
    !pip install plotly
    from plotly import graph_objects as go

In [ ]:
# plot probability distribution over characters for each timestep
fig_probs = go.Figure(
    go.Heatmap(z=stt_probs.transpose(),
               colorscale=[
                   [0, 'rgb(30,62,62)'],
                   [1, 'rgb(30,255,30)'],
               ],
               y=labels,
               dx=time_stride,
               name='Probs',
               hovertemplate='Time: %{x:.2f} s<br>Character: %{y}<br>Probability: %{z:.2f}<extra></extra>'),
    layout={
        'height': 300,
        'xaxis': {'title': 'Time, s'},
        'yaxis': {'title': 'Characters'},
        'title': 'Character Probabilities',
        'margin': dict(l=0, r=0, t=40, b=0, pad=0),
    }
)
fig_probs.show()

Extract timestamps for 'space' tokens

In [ ]:
print(np.argmax(stt_probs[6]))

In [ ]:
print(stt_probs[0])

In [ ]:
#for i in range(0, stt_probs.shape[0]):
#    print(np.argmax(stt_probs[i]))

In [ ]:
# get timestamps for space symbols
spaces = []

state = ''
idx_state = 0

if np.argmax(stt_probs[0]) == 0:
    state = 'space'

for idx in range(1, stt_probs.shape[0]):
    current_char_idx = np.argmax(stt_probs[idx])
    if state == 'space' and current_char_idx != 0 and current_char_idx != 28:
        spaces.append([idx_state, idx-1])
        state = ''
    if state == '':
        if current_char_idx == 0:
            state = 'space'
            idx_state = idx

if state == 'space':
    spaces.append([idx_state, len(stt_probs)-1])

In [ ]:
print(spaces)

In [ ]:
print(stt_output)

split audio signal into separate words

In [ ]:
from IPython.display import Audio, display

In [ ]:
signal, sample_rate = librosa.load(TEST_FILE[0], sr=None)

In [ ]:
print(stt_output)

In [ ]:
# calibration offset for timestamps: 180 ms
offset = -0.18

# split the transcript into words
words = stt_output.split()

# cut words
pos_prev = 0
for j, spot in enumerate(spaces):
    display(words[j])
    pos_end = offset + (spot[0]+spot[1])/2*time_stride
    display(Audio(signal[int(pos_prev*sample_rate):int(pos_end*sample_rate)],
                 rate=sample_rate))
    pos_prev = pos_end

display(words[j+1])
display(Audio(signal[int(pos_prev*sample_rate):],
        rate=sample_rate))


## Language Model
Get the 3-gram language model 

In [ ]:
import gzip
import os, shutil, wget

lm_gzip_path = 'language_model/3-gram.pruned.1e-7.arpa.gz'
if not os.path.exists(lm_gzip_path):
    print('Downloading pruned 3-gram model.')
    lm_url = 'http://www.openslr.org/resources/11/3-gram.pruned.1e-7.arpa.gz'
    lm_gzip_path = wget.download(lm_url)
    print('Downloaded the 3-gram language model.')
else:
    print('Pruned .arpa.gz already exists.')

uppercase_lm_path = 'language_model/3-gram.pruned.1e-7.arpa'
if not os.path.exists(uppercase_lm_path):
    with gzip.open(lm_gzip_path, 'rb') as f_zipped:
        with open(uppercase_lm_path, 'wb') as f_unzipped:
            shutil.copyfileobj(f_zipped, f_unzipped)
    print('Unzipped the 3-gram language model.')
else:
    print('Unzipped .arpa already exists.')

lm_path = 'language_model/lowercase_3-gram.pruned.1e-7.arpa'
if not os.path.exists(lm_path):
    with open(uppercase_lm_path, 'r') as f_upper:
        with open(lm_path, 'w') as f_lower:
            for line in f_upper:
                f_lower.write(line.lower())
print('Converted language model file to lowercase.')

Instantiate beam search with language model

In [ ]:
lm_path = "language_model/wav2vec2-xls-r-1b-german-german-5g.binary"

In [ ]:
beam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(
    vocab=list(model.decoder.vocabulary),
    beam_width=16,
    alpha=2, beta=1.5,
    lm_path=lm_path,
    num_cpus=max(os.cpu_count(), 1),
    input_tensor=False)

In [ ]:
beam_search_lm.forward(log_probs = np.expand_dims(stt_probs, axis=0), log_probs_length=None)